# Text preprocessing

During text preprocessing, a corpus of documents is tokenized (i.e. the document strings are split into individual words, punctuation, numbers, etc.) and then these tokens can be transformed, filtered or annotated. The goal is to prepare the raw texts in a way that makes it easier to perform eventual analysis methods in a later stage, e.g. by reducing noise in the dataset. tmtoolkit provides a rich set of tools for this purpose in the [tmtoolkit.preprocess](api.rst#tmtoolkit-preprocess) module.   

## Two approaches: functional API and `TMPreproc` class

There are two ways to apply text preprocessing methods to your documents: First, there is the [functional API](api.rst#module-tmtoolkit.preprocess) which consists of a set of Python functions that accept a list of (tokenized) documents. An example might be:

```python
corpus = [
    "Hello world!",    # document 1
    "Another example"  # document 2
]

docs = tokenize(corpus)
to_lowercase(docs)
# Out: [['hello', 'world', '!'],
#       ['another', 'example']]
```


The advantage of this approach is that it's very straight-forward and flexible. However, you must manage any meta data associated with the documents on your own (e.g. document labels or token metadata). Furthermore, the processing is not done in parallel.

Second, there is the [TMPreproc class](api.rst#tmpreproc-class-for-parallel-text-preprocessing) which addresses these limitations. You can create an instance of this class from your (labelled) documents and then apply preprocessing methods to it. This instance is a "state-machine", i.e. its contents (the documents) can change when you call a method. An example:

```python
corpus = {
    "doc1": "Hello world!",
    "doc2": "Another example"
}

preproc = TMPreproc(corpus)     # documents are directly tokenized
preproc.tokens_to_lowercase()   # this changes the documents
preproc.tokens                  # one of many ways to access the tokens

# Out:
# {
#   'doc1': ['hello', 'world', '!'],
#   'doc2': ['another', 'example']
# }
```

The most important advantage is that `TMPreproc` employs parallel processing, i.e. it uses all available processors on your machine to do the computations necessary during preprocessing. For large text corpora, this can lead to a strong speed up. 

Both approaches offer mostly the same features in terms of available preprocessing methods. `TMPreproc` has some more methods to export the data to dataframes or datatables. In general, the functional API is mostly used for quick prototyping and when using a small amount of data. For projects with large amounts of data, it's recommended to use `TMPreproc`, especially because of the parallel computation support.

This chapter starts with a few examples using the functional API and then turns to `TMPreproc`.

## Functional API

The functions in the preprocessing module make up the [functional API](api.rst#module-tmtoolkit.preprocess) for text preprocessing. We will explore some of the available functions. Most of them require at least passing a list of tokenized documents. In order to tokenize raw text documents (for example from a [Corpus](text_corpora.ipynb) object), we can use [tokenize()](api.rst#tmtoolkit.preprocess.tokenize). 

### Loading example data

Let's load a sample of three documents from the built-in *NewsArticles* dataset. We'll save the document labels in `doc_labels` since the functional API works with lists of documents (not with dicts): 

In [1]:
import random
random.seed(20191018)   # to make the sampling reproducible

from tmtoolkit.corpus import Corpus
from tmtoolkit.preprocess import tokenize

corpus = Corpus.from_builtin_corpus('english-NewsArticles').sample(3)
doc_labels = list(corpus.keys())
doc_labels

['NewsArticles-1880', 'NewsArticles-3350', 'NewsArticles-99']

### Tokenization

We can now tokenize these documents. We use `corpus.values()` to pass a list of documents. We get a list of tokenized documents back (i.e. a list of lists). We peak into the documents by only showing the first 10 words at maximum.

In [2]:
docs = tokenize(corpus.values())
[doc[:10] for doc in docs]

[['White',
  'House',
  'aides',
  'told',
  'to',
  'keep',
  'Russia-related',
  'materials',
  'Lawyers',
  'for'],
 ['Frustration',
  'as',
  'cabin',
  'electronics',
  'ban',
  'comes',
  'into',
  'force',
  'Passengers',
  'decry'],
 ['Should',
  'you',
  'have',
  'two',
  'bins',
  'in',
  'your',
  'bathroom',
  '?',
  'Our']]

### Corpus language

Some preprocessing steps are language-dependent, i.e. they're trained for different languages and hence you have to tell in which language your documents are written. At the moment, tmtoolkit only supports two languages off the shelf: English and German. 

In the functional API, all functions that are language-dependent have a `language` argument. Examples of such functions are [tokenize()](api.rst#tmtoolkit.preprocess.tokenize), [pos_tag()](api.rst#tmtoolkit.preprocess.pos_tag), [stem()](api.rst#tmtoolkit.preprocess.stem) and [lemmatize()](api.rst#tmtoolkit.preprocess.lemmatize). The default language for the `language` parameter of the preprocessing functions is set in [tmtoolkit.defaults.language](api.rst#tmtoolkit.defaults.language). If you don't change it, it's set to `"english"`. So you have two options when you use the functional API and work with a corpus that is not in English: you either or pass the `language` parameter each time you use a language-dependent function; or you set `tmtoolkit.defaults.language` right at the beginning which will be used as default for all further language-dependent preprocessing functions. Let's try both options with a German sample corpus:

In [3]:
from tmtoolkit.preprocess import stem

docs_de = [
    'Von der Wiege bis zur Bahre, Formulare, Formulare.',
    'Fischers Fritz fischt frische Fische.',
    'Viel schon ist getan, mehr noch ist zu tun, sagt der Wasserhahn zum Wasserhuhn.'
]

Option 1, passing the `language` parameter each time:

In [4]:
tokens_de = tokenize(docs_de, language='german')
stemmed_de = stem(tokens_de, language='german')
stemmed_de

[['von',
  'der',
  'wieg',
  'bis',
  'zur',
  'bahr',
  ',',
  'formular',
  ',',
  'formular',
  '.'],
 ['fisch', 'fritz', 'fischt', 'frisch', 'fisch', '.'],
 ['viel',
  'schon',
  'ist',
  'getan',
  ',',
  'mehr',
  'noch',
  'ist',
  'zu',
  'tun',
  ',',
  'sagt',
  'der',
  'wasserhahn',
  'zum',
  'wasserhuhn',
  '.']]

Option 2, setting `tmtoolkit.defaults.language` provides the same output:

In [5]:
import tmtoolkit.defaults
tmtoolkit.defaults.language = 'german'

tokens_de = tokenize(docs_de)
stemmed_de == stem(tokens_de) 

True

We will return to the English corpus hence we can reset the default language and clean up:

In [6]:
tmtoolkit.defaults.language = 'english'

del docs_de, tokens_de, stemmed_de 

### A small tour around the functional preprocessing API

We will continue with the most important functions in the preprocessing API and apply them to our English sample corpus.

#### Document length

The document length is the number of tokens per document and can be obtained with [doc_lengths()](api.rst#tmtoolkit.preprocess.doc_lengths):

In [7]:
from tmtoolkit.preprocess import doc_lengths

doc_lengths(docs)

[227, 646, 1052]

#### Vocabulary and document frequencies

The vocabulary is the set of unique tokens in the corpus, i.e. all tokens that occur at least once in at least one of the documents. You can use [vocabulary()](api.rst#tmtoolkit.preprocess.vocabulary) for that and [vocabulary_counts()](api.rst#tmtoolkit.preprocess.vocabulary_counts) to additionally get the number of times each token appears in the corpus. 

The document frequency of a token is the number of documents in which this token occurs at least once. The function [doc_frequencies()](api.rst#tmtoolkit.preprocess.doc_frequencies) returns this measure for all tokens in the vocabulary. 

In [8]:
from tmtoolkit.preprocess import vocabulary, vocabulary_counts, doc_frequencies

# first 10 entries from the sorted vocab
vocabulary(docs, sort=True)[:10]

['%', "'", "''", "'s", '(', ')', ',', '-', '-Al', '.']

In [9]:
# get unsorted vocabulary counts as Counter object
vocab_counts = vocabulary_counts(docs)
# get top 10 tokens by occurrence
vocab_counts.most_common(10)

[('the', 82),
 (',', 70),
 ('.', 60),
 ('to', 53),
 ('and', 45),
 ('in', 38),
 ('a', 31),
 ('``', 28),
 ('of', 25),
 ("''", 23)]

In [10]:
doc_freq = doc_frequencies(docs)

# "the" occurs in all three documents, "Lawyers" only in one
doc_freq['the'], doc_freq['Lawyers']


(3, 1)

#### Part-of-speech (POS) tagging

Part-of-speech (POS) tagging finds the grammatical word-category for each token in a document. The function [pos_tag()](api.rst#tmtoolkit.preprocess.pos_tag) employs this for the whole corpus. It returns a list of tags for each document. These tags conform to a specific *tagset*. For English this is the [Penn Treebank tagset](https://ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html) and for German this is the [STTS tagset](http://www.ims.uni-stuttgart.de/forschung/ressourcen/lexika/TagSets/stts-table.html).

These tags can be used to filter, annotate or lemmatize the documents.

Remember that this is a language-dependent function.

In [11]:
from tmtoolkit.preprocess import pos_tag

docs_pos = pos_tag(docs)

# show pairs of tokens and POS tags for the first 10 tokens in the first document
list(zip(docs[0][:10], docs_pos[0][:10]))

[('White', 'NNP'),
 ('House', 'NNP'),
 ('aides', 'NNS'),
 ('told', 'VBD'),
 ('to', 'TO'),
 ('keep', 'VB'),
 ('Russia-related', 'JJ'),
 ('materials', 'NNS'),
 ('Lawyers', 'NNS'),
 ('for', 'IN')]

#### Stemming and lemmatization

Stemming and lemmatization bring a token, if it is a word, to a base form. The former method is rule-based and creates base forms by cutting of common pre- and suffixes. The resulting token may not be a lexicographically correct word any more. We've already used [stem()](api.rst#tmtoolkit.preprocess.stem) in an example above.

Lemmatization is a more sophisticated process that tries to find lexicographically correct base form of a given word by also considering its POS tag and possibly its context (tokens and POS tags nearby). It's usually not rule-based but a trained model that predicts the base form from the mentioned parameters. Lemmatization can be applied with [lemmatize()](api.rst#tmtoolkit.preprocess.lemmatize).

Remember that both functions are language-dependent.

In [12]:
from tmtoolkit.preprocess import lemmatize

docs_lem = lemmatize(docs, docs_pos)
# show pairs of original tokens and lemmata for the first 10 tokens of first document
list(zip(docs[0][:10], docs_lem[0][:10]))

[('White', 'White'),
 ('House', 'House'),
 ('aides', 'aide'),
 ('told', 'tell'),
 ('to', 'to'),
 ('keep', 'keep'),
 ('Russia-related', 'Russia-related'),
 ('materials', 'material'),
 ('Lawyers', 'Lawyers'),
 ('for', 'for')]

#### "Cleaning" tokens

Depending on your methodology, it may be necessary to "clean" or "normalize" your tokens in different ways in order to remove noise from the corpus, such as punctuation tokens or numbers, upper/lowercase forms of the same word, etc. Note that this is usually not necessary when you work with more modern approaches such as word embeddings (word vectors).   

If you want to remove certain characters in *all* tokens in your corpus, you can use [remove_chars()](api.rst#tmtoolkit.preprocess.remove_chars) and pass it a sequence of characters to remove.

Note that for the following examples I continue working with the lemmatized documents `docs_lem`.

In [13]:
from tmtoolkit.preprocess import remove_chars

# remove all vowels from the documents, show first 10 tokens from first document
remove_chars(docs_lem, 'aeiou')[0][:10]

['Wht', 'Hs', 'd', 'tll', 't', 'kp', 'Rss-rltd', 'mtrl', 'Lwyrs', 'fr']

You can for example use this to remove all punctuation characters from all tokens:

In [14]:
import string

docs_clean = remove_chars(docs_lem, string.punctuation)
# show pairs of original tokens and cleaned tokens for the first 10 tokens of 2nd doc.
list(zip(docs_lem[2][:10], docs_clean[2][:10]))

[('Should', 'Should'),
 ('you', 'you'),
 ('have', 'have'),
 ('two', 'two'),
 ('bin', 'bin'),
 ('in', 'in'),
 ('your', 'your'),
 ('bathroom', 'bathroom'),
 ('?', ''),
 ('Our', 'Our')]

Notice how the token `'?'` was transformed to an empty string `''`, because "?" is a punctuation character.

A common (but harsh) practice is to transform all tokens to lowercase forms, which can be done with :

In [15]:
from tmtoolkit.preprocess import to_lowercase

docs_clean = to_lowercase(docs_clean)
docs_clean[2][:10]

['should', 'you', 'have', 'two', 'bin', 'in', 'your', 'bathroom', '', 'our']

The function [clean_tokens()](api.rst#tmtoolkit.preprocess.clean_tokens) finally applies several steps that remove tokens that meet certain criteria. This includes removing:

- punctuation tokens
- stopwords (very common words for the given language)
- empty tokens (i.e. `''`)
- tokens that are longer or shorter than a certain number of characters
- numbers  

Note that this is a language-dependent function, because the default stopword list is determined per language. This function has lot's of parameters to tweak, so it's recommended to check out the documentation.

In [16]:
from tmtoolkit.preprocess import clean_tokens

# remove punct., stopwords, empty tokens (this is the default)
# plus tokens shorter than 2 characters and numeric tokens like "2019"
docs_final = clean_tokens(docs_clean, remove_shorter_than=2, remove_numbers=True)

# first 10 tokens of doc. #2
docs_final[2][:10]

['two',
 'bin',
 'bathroom',
 'bathroom',
 'fill',
 'shampoo',
 'bottle',
 'toilet',
 'roll',
 'cleaning']

Due to the removal of several tokens in the previous step, the document lengths for the processed corpus are much smaller than for the original corpus:

In [17]:
doc_lengths(docs), doc_lengths(docs_final)

([227, 646, 1052], [129, 310, 504])

We can also observe that the vocabulary got smaller after the processing steps, which, for large corpora, is also important in terms of computation time and memory consumption for later analyses:

In [18]:
len(vocabulary(docs)), len(vocabulary(docs_final))

(681, 478)

You can also apply custom token transform functions by using [transform()](api.rst#tmtoolkit.preprocess.transform) and passing it a function that should be applied to each token in each document (hence it must accept one string argument).

First let's define such a function. Here we create a simple function that should return a token's "shape" in terms of the case of its characters:

In [19]:
def token_shape(t):
    return ''.join(['X' if str.isupper(c) else 'x' for c in t])

token_shape('USA'), token_shape('CamelCase'), token_shape('lower')

('XXX', 'XxxxxXxxx', 'xxxxx')

We can now apply this function to our corpus:

In [20]:
from tmtoolkit.preprocess import transform

doc_shapes = transform(docs, token_shape)

# show pairs of tokens and POS tags for the first 10 tokens in the first document
list(zip(docs[0][:10], doc_shapes[0][:10]))

[('White', 'Xxxxx'),
 ('House', 'Xxxxx'),
 ('aides', 'xxxxx'),
 ('told', 'xxxx'),
 ('to', 'xx'),
 ('keep', 'xxxx'),
 ('Russia-related', 'Xxxxxxxxxxxxxx'),
 ('materials', 'xxxxxxxxx'),
 ('Lawyers', 'Xxxxxxx'),
 ('for', 'xxx')]

#### Keywords-in-context (KWIC)

*Keywords-in-context (KWIC)* allow you to quickly to investigate certain keywords and their neighborhood of tokens, i.e. the tokens that appear right before and after this keyword.

tmtoolkit provides two functions for this purpose:

- [kwic()](api.rst#tmtoolkit.preprocess.kwic) is the base function accepting the input documents, a search pattern and several options that control how the search pattern is matched (more on that below); use this function when you want to further process the output of a KWIC search;
- [kwic_table()](api.rst#tmtoolkit.preprocess.kwic_table) is the more "user friendly" version of the above function as it produces a datatable with the highlighted keyword by default

Let's see both functions in action:

In [21]:
from tmtoolkit.preprocess import kwic, kwic_table

kwic(docs, 'news')

[[],
 [['told', 'Reuters', 'news', 'agency', '.'],
  ['Jazeera', 'and', 'news', 'agencies']],
 []]

We see that the first and last document do not contain any keyword that matches `"news"`, hence we get empty results for these documents. In the second document, we get two result contexts for the requested keyword. This keyword stands in the middle and is surrounded by its "context tokens", which by default means two tokens to the left and two tokens to the right. Notice that in the second result context only one token to the right is shown since the document ends after "agencies".

In [22]:
kwic_table(docs, 'news')

,doc,context,kwic
,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪
0,1,0,told Reuters *news* agency .
1,1,1,Jazeera and *news* agencies


With `kwic_table()`, we get back a datatable which provides a better formatting for quick investigation. See how the matched tokens are highlighted as `*news*` and empty results are removed (only document "1" contains the keyword which is the *second* document – remember that Python indexing starts with 0).

We can also pass the document labels via `doc_labels` to get proper labels in the `doc` column instead of document indices:

In [23]:
kwic_table(docs, 'news', doc_labels=doc_labels)

,doc,context,kwic
,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪
0,NewsArticles-3350,0,told Reuters *news* agency .
1,NewsArticles-3350,1,Jazeera and *news* agencies


Another important parameter is `context_size`. It determines the number of tokens to display left and right to the found keyword. You can either pass a single integer for a symmetric context or a tuple with integers `(<left>, <right>)`.

In [24]:
# a symmetric context of size (5, 5)
kwic_table(docs, 'news', context_size=5, doc_labels=doc_labels)

,doc,context,kwic
,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪
0,NewsArticles-3350,0,"a traveler , told Reuters *news* agency . Al Jazee…"
1,NewsArticles-3350,1,Source : -Al Jazeera and *news* agencies


In [25]:
# an asymmetric context of size (5, 1)
kwic_table(docs, 'news', context_size=(5, 1), doc_labels=doc_labels)

,doc,context,kwic
,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪
0,NewsArticles-3350,0,"a traveler , told Reuters *news* agency"
1,NewsArticles-3350,1,Source : -Al Jazeera and *news* agencies


The KWIC functions become really powerful when using the pattern matching options. So far, we were looking for *exact* matches between the corpus tokens and our keyword `"news"`. However, it is also possible to match patterns like `"new*"` (matches any word starting with "new") or `"agenc(y|ies)"` (a regular expression matching "agency" and "agencies"). The next section gives an introduction on the different options for pattern matching.

#### Common parameters for pattern matching functions

Several functions and methods in tmtoolkit support pattern matching, including the already mentioned KWIC functions but also functions for filtering tokens or documents as you will see later. They all share similar function signatures, i.e. similar parameters:

- `search_token` or `search_tokens`: allows to specify one or more patterns as strings
- `match_type`: sets the matching type and can be one of the following options:
  - `'exact'` (default): exact string matching (optionally ignoring character case), i.e. no pattern matching
  - `'regex'` uses [regular expression](https://docs.python.org/3/library/re.html) matching
  - `'glob'` uses "glob patterns" like `"politic*"` which matches for example "politic", "politics" or "politician" (see [globre package](https://pypi.org/project/globre/))
- `ignore_case`: ignore character case (applies to all three match types)
- `glob_method`: if `match_type` is 'glob', use this glob method. Must be `'match'` or `'search'` (similar behavior as Python's [re.match](https://docs.python.org/3/library/re.html#re.match) or [re.search](https://docs.python.org/3/library/re.html#re.search)
- `inverse`: inverse the match results, i.e. if matching for "hello", return all results that do *not* match "hello"

Let's try out some of these options with `kwic_table()`:

In [26]:
# using a regular expression, ignoring case
kwic_table(docs, r'agenc(y|ies)', match_type='regex', ignore_case=True,
           doc_labels=doc_labels)

,doc,context,kwic
,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪
0,NewsArticles-1880,0,law enforcement *agencies* to keep
1,NewsArticles-1880,1,"organizations , *agencies* and individuals"
2,NewsArticles-3350,0,Reuters news *agency* . Al
3,NewsArticles-3350,1,and news *agencies*


In [27]:
# using a glob, ignoring case
kwic_table(docs, 'pol*', match_type='glob', ignore_case=True,
           doc_labels=doc_labels)

,doc,context,kwic
,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪
0,NewsArticles-1880,0,false and *politically* motivated attacks
1,NewsArticles-99,0,", senior *policy* adviser for"


In [28]:
# using a glob, ignoring case
kwic_table(docs, '*sol*', match_type='glob', ignore_case=True,
           doc_labels=doc_labels)

,doc,context,kwic
,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪
0,NewsArticles-99,0,potential simple *solution* that could
1,NewsArticles-99,1,confused by *aerosols* . ''
2,NewsArticles-99,2,"bottles , *aerosols* for deodorant"


In [29]:
# using a regex that matches all tokens with at least one vowel and
# inverting these matches, i.e. all tokens *without* any vowels
kwic_table(docs, r'[AEIOUaeiou]', match_type='regex', inverse=True,
           doc_labels=doc_labels)

,doc,context,kwic
,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪
0,NewsArticles-1880,0,in the *2016* presidential election
1,NewsArticles-1880,1,"related investigations *,* ABC News"
2,NewsArticles-1880,2,has confirmed *.* `` The
3,NewsArticles-1880,3,confirmed . *``* The White
4,NewsArticles-1880,4,"motivated attacks *,* '' an"
5,NewsArticles-1880,5,"attacks , *''* an administration"
6,NewsArticles-1880,6,News Wednesday *.* The directive
7,NewsArticles-1880,7,last week *by* Senate Democrats
8,NewsArticles-1880,8,"between Trump *'s* administration ,"


#### Filtering tokens and documents

We can use the pattern matching parameters in numerous filtering functions and methods. The heart of many of these functions is [token_match()](api.rst#tmtoolkit.preprocess.token_match). Given a search pattern, a list of tokens and optional some pattern matching parameters, it returns a binary NumPy array of the same length as the input tokens. Each occurrence of `True` in this binary array signals a match.


In [30]:
from tmtoolkit.preprocess import token_match

doc0_snippet = docs[0][:10]   # first 10 tokens of first doc.
# get all tokens that match "to*"
matches = token_match('to*', doc0_snippet, match_type='glob')

# iterate through tokens and matches, show pair-wise results
for tok, match in zip(doc0_snippet, matches):
    print(tok, ':', match)

White : False
House : False
aides : False
told : True
to : True
keep : False
Russia-related : False
materials : False
Lawyers : False
for : False


The `token_match()` function is a rather low-level function that you may use for pattern matching against any list/array of strings, e.g. a list of tokens, file names, etc.

The following functions cover common use-cases for filtering during text preprocessing. Many of these functions start either with `filter_...()` or `remove_...()` and these pairs of filter and remove functions are complements. A filter function will always *retain* the matched elements whereas a remove function will always *drop* the matched elements. We can observe that with the first pair of functions, [filter_tokens()](api.rst#tmtoolkit.preprocess.filter_tokens) and [remove_tokens()](api.rst#tmtoolkit.preprocess.remove_tokens):

In [31]:
from tmtoolkit.preprocess import filter_tokens, remove_tokens

# retain only the tokens that match the pattern in each document
filter_tokens(docs, '*house*', match_type='glob', ignore_case=True)

[['House', 'House', 'House', 'House'],
 [],
 ['house', 'greenhouse', 'household']]

In [32]:
# retain only the tokens that DON'T match the pattern in each document
# will only show the first 10 tokens from the first document here, b/c
# the resulting documents are too long; you can see that "House" was
# removed from ["White", "House", ...]
remove_tokens(docs, '*house*', match_type='glob', ignore_case=True)[0][:10]

['White',
 'aides',
 'told',
 'to',
 'keep',
 'Russia-related',
 'materials',
 'Lawyers',
 'for',
 'the']

The pair [filter_documents()](api.rst#tmtoolkit.preprocess.filter_documents) and [remove_documents()](api.rst#tmtoolkit.preprocess.remove_documents) works similarily, but filters or drops whole documents regarding the supplied match criteria. Both accept the standard pattern matching parameters but also a parameter `matches_threshold` with default value `1`. When this number of matching tokens is hit, the document will be part of the result set (`filter_documents()`) or removed from the result set (`remove_documents()`). By this, we can for example retain only those documents that contain certain token patterns.

Let's try these functions out in practice. This time I will also pass the `doc_labels` so that the filtering also applies to our list of document labels. If `doc_labels` is also passed, the functions return two results – the filtered list of documents and the filtered list of document labels.

In [33]:
from tmtoolkit.preprocess import filter_documents, remove_documents

filtered_docs, filtered_doc_labels = filter_documents(docs, '*house*',
                                                      doc_labels=doc_labels,
                                                      match_type='glob',
                                                      ignore_case=True)
filtered_doc_labels

['NewsArticles-1880', 'NewsArticles-99']

We can see that two out of three documents contained the pattern `'*house*'` and hence were retained. The list `filtered_docs` represents these two documents (I don't print them here because they are too long).

We can also adjust `matches_threshold` to set the minimum number of token matches for filtering:

In [34]:
filtered_docs, filtered_doc_labels = filter_documents(docs, '*house*',
                                                      doc_labels=doc_labels,
                                                      match_type='glob',
                                                      ignore_case=True,
                                                      matches_threshold=4)
filtered_doc_labels

['NewsArticles-1880']

In [35]:
filtered_docs, filtered_doc_labels = remove_documents(docs, '*house*',
                                                      doc_labels=doc_labels,
                                                      match_type='glob',
                                                      ignore_case=True)
filtered_doc_labels

['NewsArticles-3350']

When we use `remove_documents()` we get only these documents, that did *not* contain the specified pattern.

Another useful pair of functions is [filter_documents_by_name()](api.rst#tmtoolkit.preprocess.filter_documents_by_name) and [remove_documents_by_name()](api.rst#tmtoolkit.preprocess.remove_documents_by_name) which again accepts the same pattern matching parameters but only applies them to the document names, i.e. document *labels*:

In [36]:
from tmtoolkit.preprocess import filter_documents_by_name

filtered_docs, filtered_doc_labels = filter_documents_by_name(docs, doc_labels,
                                                              r'-\d{4}$',
                                                              match_type='regex')
filtered_doc_labels

['NewsArticles-1880', 'NewsArticles-3350']

In the above example we wanted to retain only these documents, whose document labels ended with exactly 4 digits like "...-1234". Hence, we only get "NewsArticles-1880" and "NewsArticles-3350" but not "NewsArticles-99". Again, `remove_documents_by_name()` will do the exact opposite.

When you annotated your documents' tokens with Part-of-Speech (POS) tags, you can also filter them using [filter_for_pos()](api.rst#tmtoolkit.preprocess.filter_for_pos). You need to pass the documents, the POS tags for them and the POS tag(s) to be used for filtering:

In [38]:
from tmtoolkit.preprocess import filter_for_pos

filtered_docs, filtered_docs_pos = filter_for_pos(docs, docs_pos, 'N')
# displaying only the first 10 filtered tokens from the first document
filtered_docs[0][:10]

['White',
 'House',
 'aides',
 'materials',
 'Lawyers',
 'Trump',
 'administration',
 'White',
 'House',
 'aides']

In this example we filtered for tokens that were identified as nouns by passing the *simplified POS tag* `'N'` (for more on simplified tags, see the function documentation). We can also filter for more than one tag, e.g. nouns or verbs:

In [40]:
filtered_docs, filtered_docs_pos = filter_for_pos(docs, docs_pos, ['N', 'V'])
# displaying only the first 10 filtered tokens from the first document
filtered_docs[0][:10]

['White',
 'House',
 'aides',
 'told',
 'keep',
 'materials',
 'Lawyers',
 'Trump',
 'administration',
 'have']

`filter_for_pos()` has no `remove_...()` counterpart, but you can set the `inverse` parameter to `True` to achieve the same effect.

Finally there are two functions for removing tokens based on their [document frequency](#Vocabulary-and-document-frequencies): [remove_common_tokens()](api.rst#tmtoolkit.preprocess.remove_common_tokens) and [remove_uncommon_tokens()](api.rst#tmtoolkit.preprocess.remove_uncommon_tokens). The former removes all tokens that have a document frequency greater or equal a certain threshold defined by parameter `df_threshold`. The latter does the same for all tokens that have a document frequency lower or equal `df_threshold`. This parameter can either be a relative (default) frequency or absolute count (by setting parameter `absolute=True`).

Before applying the function, let's have a look at the number of tokens per document again, to later see how many we will remove:

In [45]:
doc_lengths(docs)

[227, 646, 1052]

In [53]:
from tmtoolkit.preprocess import remove_common_tokens

doc_lengths(remove_common_tokens(docs, df_threshold=0.9))

[143, 413, 699]

By removing all tokens with a document threshold of at least 0.9, we would remove quite a number of tokens in each document. Let's investigate the vocabulary in order to see, which tokens are removed:

In [52]:
orig_vocab = vocabulary(docs)  # vocabulary of unfiltered documents

filtered_docs = remove_common_tokens(docs, df_threshold=0.9)
filtered_vocab = vocabulary(filtered_docs)
orig_vocab - filtered_vocab   # set difference gives removed vocabulary tokens

{"''",
 "'s",
 ',',
 '.',
 '?',
 'The',
 '``',
 'a',
 'all',
 'also',
 'an',
 'and',
 'be',
 'for',
 'has',
 'have',
 'in',
 'into',
 'is',
 'more',
 'of',
 'on',
 'or',
 'other',
 'such',
 'than',
 'that',
 'the',
 'to',
 'which',
 'with'}

`remove_uncommon_tokens` works similarily. This time, let's use an absolute number as threshold:

In [59]:
from tmtoolkit.preprocess import remove_uncommon_tokens

filtered_docs = remove_uncommon_tokens(docs, df_threshold=1, absolute=True)
filtered_vocab = vocabulary(filtered_docs)
# set difference gives removed vocabulary tokens
# this time, show only the first 10 tokens that were removed
sorted(orig_vocab - filtered_vocab)[:10]

['%', '(', ')', '-Al', '.-', '10', '12', '135,000', '2016', '38']

The above means that we remove all tokens that appear only in exactly one document.

#### Expanding contractions and "gluing" tokens

#### Generating n-grams

#### Generating a sparse document-term matrix (DTM)

